In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from shapely.geometry import LineString

import pydeck as pdk
import pandas as pd
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

In [ ]:
directory = 'traffic_data_sample/traffic_data_sample/traffic_data_sample.shp'
gdf = gpd.read_file(directory)

In [ ]:

county = 12057
lines = gdf.query(f"county_fip == {county}").head(65000)
    
# List to hold the line data for Pydeck
line_data = []

# Create the line data from the geometries
for line in lines['geometry']:
    x, y = line.xy
    x, y = np.array(x), np.array(y)
    
    # Prepare the coordinates for Pydeck
    line_coords = [[a, b] for a, b in zip(x, y)] 
    line_data.append(line_coords)

colors = []
lines["log_trips_volu"] = np.log(lines["trips_volu"]+1)
for trip_volume in lines["log_trips_volu"]:
        
    # Normalize the 'trips_volu' to a color range (e.g., using a color map)
    norm = Normalize(vmin=lines['log_trips_volu'].min(), vmax=lines['log_trips_volu'].max())
    colormap = ScalarMappable(norm=norm, cmap="viridis")  # You can choose a different colormap
    color = colormap.to_rgba(trip_volume)[:3]  # Extract RGB values
    colors.append([int(c * 255) for c in color])  # Convert to 0-255 range for Pydeck

# Create DataFrame for the paths
df = pd.DataFrame({
    'path_id': list(lines["id"]),
    'path': line_data,
    'color':colors, #[[255, 0, 0] for _ in range(len(lines))]   # Red and Blue paths,
    'volume':lines["trips_volu"]
})

# Define the layer
layer = pdk.Layer(
    'PathLayer',
    df,
    get_path='path',
    get_color='color',
    width_scale=20,
    width_min_pixels=2,
    pickable=True,
    auto_highlight=True
)

# Set the initial view
view_state = pdk.ViewState(
    latitude=y[0],
    longitude=x[0],
    zoom=10,
    pitch=45,
    bearing=0
)

# Create the deck
deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={'text': 'Path: {path_id} {volume}'}
)

# Show the deck
deck.show()

# If you want to save it as HTML
deck.to_html('linestring_visualization.html')